In [62]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

from bs4 import BeautifulSoup
import requests


In [63]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html.parser')

#print(soup.prettify())
#soup.title.string
#soup.find_all("tr")


In [64]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
Toronto_nbh = pd.DataFrame(columns=column_names)

In [65]:
for data in soup.find_all('tr'):
    data_1 = data.find('td')
#    print(data_1)
    if data_1 != None and data_1.string != None:
        if data_1.string[0] == 'M':
            postalcode = data_1.string
#            print(postalcode)
            
            data_2 = data_1.next_sibling.next_sibling
#            print(data_2)
            try:
                borough = data_2.find('a').text
#                print(borough)

                data_3 = data_2.next_sibling.next_sibling
#                print(data_3)
                try:
                    neighborhood = data_3.find('a').text
                except Exception as e:
                    neighborhood = borough  #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
#                print(neighborhood)
            
                Toronto_nbh = Toronto_nbh.append({'PostalCode': postalcode,
                                                  'Borough': borough,
                                                  'Neighborhood': neighborhood}, ignore_index=True)
            
            except Exception as e:
                borough = data_2.string  #Ignore cells with a borough that is Not assigned.
                            


In [66]:
Toronto_nbh.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [67]:
def nstrcat(arr):
    return ','.join(arr)

Toronto_nbh = Toronto_nbh.groupby(['PostalCode','Borough'], as_index=False).agg(nstrcat)

In [68]:
Toronto_nbh.head(100)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Scarborough,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Scarborough
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Scarborough,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough"
9,M1N,Scarborough,"Birch Cliff,Scarborough"


In [69]:
Toronto_nbh.shape

(100, 3)

In [76]:
#Get the geographical coordinates of each postal code
df_coords = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
df_coords.shape

(103, 3)

In [78]:
#Merge the two 
Toronto_nbh.join(df_coords.set_index('Postal Code'), on='PostalCode')

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Scarborough,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Scarborough,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Scarborough,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Scarborough",43.692657,-79.264848
